<a href="https://colab.research.google.com/github/ShogoNoguchi/task_CSL/blob/main/make_SEEDV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

make_SEEDV.py
作：群馬大学　野口翔伍

エラーハンドリングが一見冗長ととれますが、これをしないと、他の方がこのコードで検証をする際、アノテーションを読み取れずエラーとなりる恐れが大きいです。

In [1]:
# DagsHubとboto3のインストール
%pip install -q dagshub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.2/252.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.8 MB/s eta 0:00:00


In [2]:
import dagshub.colab
DAGSHUB_REPO = dagshub.colab.login()

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4293ed82-e4b7-454a-9695-e16319060198&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=737d74a39e0c78e56692499de0b243bbbb87cab03c6a402e430ec6ac9ac38aef&referrer=colab




Accessing as t220g139

Repository t220g139/dagshub-drive is ready for use with Colab. Link to the repository:
https://dagshub.com/t220g139/dagshub-drive


In [3]:
mount_path = dagshub.storage.mount(DAGSHUB_REPO)

Do you want to install all missing packages?
Are you sure [y/(N)]: y


Installing rclone...

rclone installed successfully.

Installing fusermount3...

fusermount3 installed successfully.

Successfully mounted DagsHub Storage in 'dagshub-drive' to 'dagshub-drive/dagshub_storage'.

To unmount, run `dagshub.storage.unmount(repo="t220g139/dagshub-drive", path="dagshub-drive/dagshub_storage")`.

In [4]:

# MNE-Python バージョン"0.23.0"をインストール
!pip install mne==0.23.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.8 MB/s eta 0:00:00


In [5]:


# 必要なライブラリのインポート
import mne
import numpy as np
import os
import pickle
import glob
import logging
from datetime import datetime, timezone

print(mne.__version__)


0.23.0


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


以下のコードでDeprecationWarningsが発生してますが、前処理には影響がありません。BCMIが求めるmneのバージョンが低いために発生してます。

一応、警告を表示させてますが、お試しの際無くしたければ（推奨される方法ではありませんが、）
コードの先頭で

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

とします。

In [7]:


# ディレクトリの設定
raw_dir = "/content/drive/MyDrive/SEEDV/EEG_raw"
output_dir = "/content/processed/"

# 出力ディレクトリの作成
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")

for folder in ["processed_train", "processed_eval", "processed_test"]:
    folder_path = os.path.join(output_dir, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")

# ログ設定
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# ログフォーマットの設定
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# ファイルハンドラの作成
log_file_path = os.path.join(output_dir, 'processing.log')
file_handler = logging.FileHandler(log_file_path)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)

# ストリームハンドラの作成（コンソール出力）
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
stream_handler.setFormatter(formatter)

# ハンドラをロガーに追加（重複追加を防ぐ）
if not logger.handlers:
    logger.addHandler(file_handler)
    logger.addHandler(stream_handler)

# 不要なチャンネルのリスト
useless_ch = ['M1', 'M2', 'VEO', 'HEO', 'EKG', 'ECG']

# 標準チャンネル順序
chOrder_standard = [
    'FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1',
    'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6',
    'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ',
    'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2'
]

# トライアル境界（秒単位）
session_starts = {
    1: [30, 132, 287, 555, 773, 982, 1271, 1628, 1730, 2025, 2227, 2435, 2667, 2932, 3204],
    2: [30, 299, 548, 646, 836, 1000, 1091, 1392, 1657, 1809, 1966, 2186, 2333, 2490, 2741],
    3: [30, 353, 478, 674, 825, 908, 1200, 1346, 1451, 1711, 2055, 2307, 2457, 2726, 2888]
}
session_ends = {
    1: [102, 228, 524, 742, 920, 1240, 1568, 1697, 1994, 2166, 2401, 2607, 2901, 3172, 3359],
    2: [267, 488, 614, 773, 967, 1059, 1331, 1622, 1777, 1908, 2153, 2302, 2428, 2709, 2817],
    3: [321, 418, 643, 764, 877, 1147, 1284, 1418, 1679, 1996, 2275, 2425, 2664, 2857, 3066]
}

# セッションごとの感情ラベル
emotions_session1 = ["happy", "fear", "neutral", "sad", "disgust", "happy", "fear", "neutral", "sad", "disgust", "happy", "fear", "neutral", "sad", "disgust"]
emotions_session2 = ["sad", "fear", "neutral", "disgust", "happy", "happy", "disgust", "neutral", "sad", "fear", "neutral", "happy", "fear", "sad", "disgust"]
emotions_session3 = ["sad", "fear", "neutral", "disgust", "happy", "happy", "disgust", "neutral", "sad", "fear", "neutral", "happy", "fear", "sad", "disgust"]

# 感情ラベルのマッピング
emo2id = {"disgust": 0, "fear": 1, "sad": 2, "neutral": 3, "happy": 4}

# .cntファイルの取得
cnt_files = glob.glob(os.path.join(raw_dir, "*.cnt"))

# サンプリング周波数
sfreq = 200

# スキップされたファイルを記録するリスト
skipped_files = []

# 処理済みファイル数のカウント
processed_files_count = 0

# スクリプトの開始をログに記録
logger.info("EEGデータ処理を開始します。")

for cnt_file in cnt_files:
    fname = os.path.basename(cnt_file)
    parts = fname.split('_')

    # ファイル名から被験者IDとセッションIDを解析
    if len(parts) < 3:
        logger.warning(f"Filename {fname} does not conform to expected format. Skipping this file.")
        skipped_files.append(fname)
        continue

    subject_id = parts[0]
    try:
        ses_id = int(parts[1])
    except ValueError:
        logger.warning(f"Session ID in filename {fname} is not an integer. Skipping this file.")
        skipped_files.append(fname)
        continue

    # 日付部分を抽出（例：2_3_20180425.cnt -> 20180425）
    date_str = parts[-1].split('.')[0]
    try:
        meas_datetime = datetime.strptime(date_str, '%Y%m%d')
    except ValueError as e:
        logger.warning(f"Date parsing failed for {fname}: {e}. Setting meas_date to None.")
        meas_datetime = None

    # セッションIDに対応する開始・終了時間を取得
    starts = session_starts.get(ses_id)
    ends = session_ends.get(ses_id)
    if not starts or not ends:
        logger.warning(f"No session boundaries defined for session {ses_id} in {fname}. Skipping.")
        skipped_files.append(fname)
        continue

    # セッションに対応する感情ラベルの取得
    emotions = {
        1: emotions_session1,
        2: emotions_session2,
        3: emotions_session3
    }.get(ses_id, [])

    if len(emotions) != 15:
        logger.warning(f"Emotion list length mismatch for session {ses_id} in {fname}. Skipping.")
        skipped_files.append(fname)
        continue

    # .cntファイルの読み込み（date_formatを適切に処理）
    read_success = False
    for fmt in ['mm/dd/yy', 'dd/mm/yy']:
        try:
            raw = mne.io.read_raw_cnt(
                cnt_file,
                preload=True,
                date_format=fmt,
                data_format='int16',  # 必要に応じて 'auto' から 'int16' に変更
                verbose='ERROR'  # エラーメッセージのみ表示
            )
            logger.info(f"Successfully loaded {fname} with date_format='{fmt}'.")
            read_success = True
            break
        except ValueError as e:
            logger.warning(f"Failed to load {fname} with date_format='{fmt}': {e}. Trying next format.")
        except Exception as e:
            logger.error(f"Unexpected error while loading {fname} with date_format='{fmt}': {e}.")
            break  # その他のエラーの場合はループを抜ける

    if not read_success:
        logger.error(f"Failed to load {fname} with any supported date_format. Skipping this file.")
        skipped_files.append(fname)
        continue

    # 測定日が設定されていない場合、手動で設定
    if raw.info['meas_date'] is None and meas_datetime is not None:
        try:
            # meas_datetime を UTC に変換
            meas_datetime_utc = meas_datetime.replace(tzinfo=timezone.utc)
            # Unixタイムスタンプ（秒）を取得
            meas_timestamp = meas_datetime_utc.timestamp()
            raw.set_meas_date(meas_timestamp)
            logger.info(f"Set meas_date for {fname} to {meas_datetime_utc} (UTC).")
        except Exception as e:
            logger.warning(f"Failed to set meas_date for {fname}: {e}. Proceeding without it.")
    elif raw.info['meas_date'] is None and meas_datetime is None:
        logger.info(f"meas_date is not set for {fname}, proceeding without it.")

    # 不要チャンネル除去
    available_ch = raw.ch_names
    drop_ch = [ch for ch in useless_ch if ch in available_ch]
    if drop_ch:
        raw.drop_channels(drop_ch)
        logger.info(f"Dropped channels {drop_ch} from {fname}.")

    # 必要チャンネル確認
    missing_ch = [ch for ch in chOrder_standard if ch not in raw.ch_names]
    if missing_ch:
        logger.warning(f"Missing channels {missing_ch} in {fname}. Skipping this file.")
        skipped_files.append(fname)
        continue

    # チャンネル並び替え
    try:
        raw.reorder_channels(chOrder_standard)
        logger.info(f"Reordered channels for {fname}.")
    except Exception as e:
        logger.error(f"Failed to reorder channels for {fname}: {e}. Skipping this file.")
        skipped_files.append(fname)
        continue

    try:
        # フィルタリング
        logger.info(f"Filtering raw data for {fname}: Band-pass 0.1 - 75 Hz, Notch 50 Hz")
        raw.filter(l_freq=0.1, h_freq=75.0, fir_design='firwin')
        raw.notch_filter(freqs=50.0, fir_design='firwin')
        raw.resample(sfreq, n_jobs=1)
        data = raw.get_data(units='uV')
        logger.info(f"Filtered and resampled data for {fname}.")
    except Exception as e:
        logger.error(f"Filtering or resampling failed for {fname}: {e}. Skipping this file.")
        skipped_files.append(fname)
        continue

    # 各トライアルの処理
    for t_i in range(15):
        try:
            start_sample = starts[t_i] * sfreq
            end_sample = ends[t_i] * sfreq
            trial_data = data[:, start_sample:end_sample]
            y = emo2id.get(emotions[t_i], -1)
            if y == -1:
                logger.warning(f"Unknown emotion '{emotions[t_i]}' in {fname}. Skipping trial.")
                continue

            length = trial_data.shape[1]
            num_segments = length // 200

            trial_id = t_i + 1
            if trial_id <= 5:
                split_folder = "processed_train"
            elif trial_id <= 10:
                split_folder = "processed_eval"
            else:
                split_folder = "processed_test"

            for seg_i in range(num_segments):
                seg_data = trial_data[:, seg_i * 200:(seg_i + 1) * 200]
                sample_dict = {"X": seg_data, "y": y, "ch_names": chOrder_standard}
                out_name = f"{subject_id}_{ses_id}_{trial_id}_{seg_i}.pkl"
                with open(os.path.join(output_dir, split_folder, out_name), "wb") as f:
                    pickle.dump(sample_dict, f)
            logger.info(f"Processed trial {trial_id} of {fname}.")
        except IndexError as e:
            logger.error(f"Index error in trial {t_i+1} for {fname}: {e}. Skipping this trial.")
            continue
        except Exception as e:
            logger.error(f"Unexpected error in trial {t_i+1} for {fname}: {e}. Skipping this trial.")
            continue

    processed_files_count += 1
    logger.info(f"Successfully processed {fname}.")

# スキップされたファイルの一覧を出力
if skipped_files:
    logger.info("以下のファイルがスキップされました：")
    for skipped in skipped_files:
        logger.info(f"- {skipped}")
    logger.info(f"Total skipped files: {len(skipped_files)}")
else:
    logger.info("すべてのファイルが正常に処理されました。")

logger.info(f"Total processed files: {processed_files_count}")

print("All data processed for SEED-V with channel reordering.")

# 全ての処理が終了したら、DagsHubにアップロード（私が使用している機械学習用データストレージです。）
LOCAL_PATH = "/content/processed/"
PATH_IN_BUCKET = "processed/"

# アップロードコマンドの実行
!dagshub upload {DAGSHUB_REPO} {LOCAL_PATH} {PATH_IN_BUCKET} --bucket


Created directory: /content/processed/
Created folder: /content/processed/processed_train
Created folder: /content/processed/processed_eval
Created folder: /content/processed/processed_test


INFO:root:EEGデータ処理を開始します。
INFO:root:Successfully loaded 1_2_20180810.cnt with date_format='mm/dd/yy'.
INFO:root:Dropped channels ['M1', 'M2', 'VEO', 'HEO'] from 1_2_20180810.cnt.
INFO:root:Reordered channels for 1_2_20180810.cnt.
INFO:root:Filtering raw data for 1_2_20180810.cnt: Band-pass 0.1 - 75 Hz, Notch 50 Hz
/usr/local/lib/python3.10/dist-packages/mne/filter.py:1009: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  logger.info('Setting up band-pass filter from %0.2g - %0.2g Hz'
/usr/local/lib/python3.10/dist-packages/mne/filter.py:1009: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  logger.info('Setting up band-pass filter from %0.2g - 

All data processed for SEED-V with channel reordering.
Accessing as t220g139
Client created. Use the name of the repo (dagshub-drive) as the name of the bucket
  Uploading files... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 117745/117745
